In [1]:
import os
import numpy as np
import cv2  # Make sure OpenCV is imported
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [2]:
data_dir = 'demodata/asl_alphabet_train' 
categories = sorted(os.listdir(data_dir))

In [3]:
img_height, img_width = 64, 64
num_classes = len(categories)

In [4]:
images = []
labels = []

for category in categories:
    path = os.path.join(data_dir, category)
    class_num = categories.index(category) 
    for img in os.listdir(path):
        try:
            img_array = cv2.imread(os.path.join(path, img))
            img_resized = cv2.resize(img_array, (img_height, img_width))
            images.append(img_resized)
            labels.append(class_num)
        except Exception as e:
            print(f"Failed to process image {img}: {e}")
            continue

In [5]:
images = np.array(images) / 255.0
labels = to_categorical(np.array(labels), num_classes=num_classes)

In [6]:
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

In [ ]:
print(f"Number of training samples: {len(X_train)}")
print(f"Number of testing samples: {len(X_test)}")

In [8]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    
    layers.Conv2D(256, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    
    layers.Flatten(),
    
    layers.Dense(256, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    
    layers.Dense(128, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    
    layers.Dense(num_classes, activation='softmax')
])

In [10]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [11]:
datagen = ImageDataGenerator(rotation_range=15,zoom_range=0.2,width_shift_range=0.2,height_shift_range=0.2,horizontal_flip=True)
datagen.fit(X_train)

In [ ]:
history = model.fit(datagen.flow(X_train, y_train, batch_size=32),epochs=20,validation_data=(X_test, y_test))

In [13]:
model.save('sign_language_model.keras')


In [ ]:
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=2)
print(f'Test Accuracy: {test_accuracy:.2f}')

In [15]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, precision_score, recall_score
import numpy as np

In [ ]:
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)

In [ ]:
conf_matrix = confusion_matrix(y_true, y_pred_classes)
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='g', cmap='Blues', xticklabels=categories, yticklabels=categories)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
precision = precision_score(y_true, y_pred_classes, average='weighted')
print(f"Precision: {precision:.2f}")
recall = recall_score(y_true, y_pred_classes, average='weighted')

In [ ]:
recall = recall_score(y_true, y_pred_classes, average='weighted')
print(f"Recall: {recall:.2f}")
